In [2]:
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# 실험 설계

**num_words = [10000, 20000, max(30979+3)], with tokens**  
**models = [LogisticRegression, 그래디언트 부스팅 트리, 보팅]**


## 평가 기준

### 1. 분류 모델의 accuracy가 기준 이상 높게 나왔는가?	
  * 3가지 단어 개수에 대해 8가지 머신러닝 기법을 적용하여 그중 최적의 솔루션을 도출하였다. 
  
### 2. 분류 모델의 F1 score가 기준 이상 높게 나왔는가?	
  * Vocabulary size에 따른 각 머신러닝 모델의 성능변화 추이를 살피고, 해당 머신러닝 알고리즘의 특성에 근거해 원인을 분석하였다.  
  
### 3. 딥러닝 모델을 활용해 성능이 비교 및 확인되었는가?	
  * 동일한 데이터셋과 전처리 조건으로 딥러닝 모델의 성능과 비교하여 결과에 따른 원인을 분석하였다.  

In [1]:
def graph_confusion_matrix(model, x_test, y_test):#, classes_name):
    df_cm = pd.DataFrame(confusion_matrix(y_test, model.predict(x_test)))#, index=classes_name, columns=classes_name)
    fig = plt.figure(figsize=(12,12))
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=12)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=12)
    plt.ylabel('label')
    plt.xlabel('predicted value')

In [ ]:
def load_data(num_words=None):
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words, test_split=0.2)
    word_index = reuters.get_word_index(path="reuters_word_index.json")
    
    index_to_word = { index+3 : word for word, index in word_index.items() }

    for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
        index_to_word[index]=token
        
    decoded = []
    for i in range(len(x_train)):
        t = ' '.join([index_to_word[index] for index in x_train[i]])
        decoded.append(t)

    x_train = decoded

    decoded = []
    for i in range(len(x_test)):
        t = ' '.join([index_to_word[index] for index in x_test[i]])
        decoded.append(t)

    x_test = decoded
    
    dtmvector = CountVectorizer()
    tfidf_transformer = TfidfTransformer()
    
    # 데이터를 DTM으로 변환 -> DTM을 TF-IDF 행렬로 변환
    x_train_dtm = dtmvector.fit_transform(x_train)
    tfidfv_train = tfidf_transformer.fit_transform(x_train_dtm)

    x_test_dtm = dtmvector.transform(x_test)              
    tfidfv_test = tfidf_transformer.transform(x_test_dtm) 
    
    return tfidfv_train, y_train, tfidfv_test, y_test

In [ ]:
def run_models(num_words=None):
    x_train, 